# Fast Fourier Transform (FFT) derivation

In [18]:
import sympy as sp

# Define basic symbols
n, k, X_k = sp.symbols('n k X[k]', integer=True)
N = sp.Symbol('N', integer=True, positive=True)
x = sp.IndexedBase('x')
W = sp.exp(-2 * sp.pi * sp.I / N)

X_k_dft = sp.Sum(x[n] * W**(k*n), (n, 0, N-1))
print("Original DFT formula:")
display(sp.Eq(X_k, X_k_dft))

n_even = 2*n
n_odd = 2*n + 1
N_half = N//2

X_k_split = sp.Sum(x[2*n] * W**(k*2*n), (n, 0, N_half-1)) + \
            sp.Sum(x[2*n + 1] * W**(k*(2*n + 1)), (n, 0, N_half-1))

print("\nSplit into even and odd terms:")

X_k_split_outed = sp.Sum(x[2*n] * W**(k*2*n), (n, 0, N_half-1)) + \
            sp.Sum(x[2*n + 1] * sp.exp((-4 * sp.pi * sp.I * k * n - 2 * sp.pi * sp.I * k) / N), (n, 0, N_half-1))

# Simplify using W**(N/2) = -1 and W**N = 1
X_k_factored = sp.Sum(x[2*n] * (W**2)**(k*n), (n, 0, N_half-1)) + \
               W**k * sp.Sum(x[2*n + 1] * (W**2)**(k*n), (n, 0, N_half-1))
display(sp.Eq(X_k_split_outed, X_k_split))
derivateion = sp.Eq(X_k, sp.Eq(X_k_split, sp.Eq(X_k_split_outed, X_k_factored)))
display(derivateion)

Original DFT formula:


Eq(X[k], Sum(exp(-2*I*pi*k*n/N)*x[n], (n, 0, N - 1)))


Split into even and odd terms:


Eq(Sum(exp((-4*I*pi*k*n - 2*I*pi*k)/N)*x[2*n + 1], (n, 0, floor(N/2) - 1)) + Sum(exp(-4*I*pi*k*n/N)*x[2*n], (n, 0, floor(N/2) - 1)), Sum(exp(-4*I*pi*k*n/N)*x[2*n], (n, 0, floor(N/2) - 1)) + Sum(exp(-2*I*pi*k*(2*n + 1)/N)*x[2*n + 1], (n, 0, floor(N/2) - 1)))

Eq(X[k], False)

## FFT Decomposition Proof

We'll show that FFT can be decomposed into:
```
X[k] = F_even[k] + W^k * F_odd[k]
```
where F_even and F_odd are FFTs of even and odd indexed elements respectively.